## Test an Amazon Bedrock Agent

The following code example shows how to invoke an Amazon Bedrock Agent using Python. This is a very simple test as does not include session state management. 

In order to run this example, you will need a running agent, and the following information: 

 - **agentId** - (An id created for the agent, this will resemble `XYZ123ABC0`.  You can find this on the `Agents` console page under `Agent overview`.)
 - **agentAliasId** - (An id created for the deployed agent version to use, this will also resemble `XYZ123ABC0` but will be different to the agentId.  You can find this on the `Agents` console page under `Aliases`.)


*Note: This code was written and tested in Jan 2024, using boto3 version 1.34.23. You can find more details about the latest AgentsforBedrockRuntime Boto3 Client here: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime.html*

In [ ]:
import boto3
import botocore

It's not uncommon for some agents to take a significant amount of time to return.  This may be caused by agents that make take many steps to create a response.  As such we extend the timeout of the standard Boto3 client as we create an instance of the Amazon Bedrock Agent Runtime client object. 

In [ ]:
config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 0}
)

bedrock_agent_runtime = boto3.client("bedrock-agent-runtime", region_name="us-west-2", config=config)

Calling `invoke_agent` on the client will result in a Botocore streaming object called `completion` that may contain one or more `chunk`s of data.  To accommodate this, as soon as we invoke the agent, you will start to parse the response and will continue to do so until the completion is complete. Then you will print the output from the Agent. 

In [ ]:
response = bedrock_agent_runtime.invoke_agent(
    agentId='XYZ123ABC0',      # Identifier for Agent
    agentAliasId='XYZ123ABC0', # Identifier for Agent Alias
    sessionId='session123',    # Identifier used for the current session
    inputText='text to send to the agent'
)

output = ""

stream = response.get('completion')
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            output += chunk.get('bytes').decode()

print(output)